In [ ]:
! pip install pytubefix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 2.8 MB/s eta 0:00:00


In [ ]:
from pytubefix import YouTube

# Define the video URL
VIDEO_URL = "https://www.youtube.com/watch?v=w6hL_dszMxk"

# Create a YouTube object
yt = YouTube(VIDEO_URL)

# Filter for only audio streams and download the first one (MP4 format)
audio_stream = yt.streams.filter(only_audio=True, file_extension="mp4").first()

# Download the audio file
audio_stream.download(filename="ytaudio.mp4")

print("Audio download completed successfully.")


Audio download completed successfully.


In [ ]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

**English ASR with HuggingSound**

In [ ]:
! pip install git+https://github.com/openai/whisper.git
! sudo apt update && sudo apt install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-fj3ra467
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-fj3ra467
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802824 sha256=e36309185b92fb3e64c118fe4b02201b66fee0d08c5eef5078521e4f9d4e82bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-e63chki8/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,

In [ ]:
!whisper "ytaudio.wav" --model medium.en

100%|█████████████████████████████████████| 1.42G/1.42G [00:30<00:00, 49.9MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

**With GUI**

In [ ]:
!pip install gradio pytubefix
!pip install git+https://github.com/openai/whisper.git
!apt-get install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-306p6wen
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-306p6wen
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.


In [ ]:
import gradio as gr
from pytubefix import YouTube
import os
import re

# Function to process YouTube link, download audio, convert to wav, and generate text using Whisper
def process_youtube_to_text(youtube_link):
    try:
        # Step 1: Extract video ID from YouTube URL for a unique filename
        video_id = re.search(r"v=([a-zA-Z0-9_-]{11})", youtube_link)
        if not video_id:
            return "Invalid YouTube link. Please provide a valid video URL."

        video_id = video_id.group(1)
        audio_file_path = f"ytaudio_{video_id}.mp4"
        wav_file_path = f"ytaudio_{video_id}.wav"
        text_file_path = f"ytaudio_{video_id}.txt"

        # Step 2: Download YouTube audio
        yt = YouTube(youtube_link)
        audio_stream = yt.streams.filter(only_audio=True, file_extension="mp4").first()
        audio_stream.download(filename=audio_file_path)

        # Confirm audio download
        print(f"Audio downloaded: {audio_file_path}")

        # Step 3: Convert mp4 to wav using FFmpeg
        os.system(f"ffmpeg -i {audio_file_path} -acodec pcm_s16le -ar 16000 {wav_file_path}")

        # Step 4: Transcribe the audio using Whisper
        os.system(f"whisper {wav_file_path} --model medium.en")

        # Step 5: Read the transcription from the text file generated by Whisper
        if os.path.exists(text_file_path):
            with open(text_file_path, 'r') as f:
                transcription = f.read()
        else:
            transcription = "Error: Transcription file not found."

        # Return the transcription
        return transcription

    except Exception as e:
        return f"Error occurred: {e}"

# Define Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# YouTube Audio Transcriber")

    # Input for YouTube link
    youtube_input = gr.Textbox(label="Enter YouTube Link", placeholder="https://www.youtube.com/watch?v=...")

    # Output box to display the transcription
    output_textbox = gr.Textbox(label="Transcribed Text", placeholder="Transcription will appear here.")

    # Button to process the audio and generate transcription
    submit_button = gr.Button("Generate Text")

    # Define the action of the button
    submit_button.click(process_youtube_to_text, youtube_input, output_textbox)

# Launch the interface
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2f845ab8c0b2522fc7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
